In [ ]:
pip install pyarrow

In [ ]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path(".").resolve()
RAW_MST_DIR = PROJECT_ROOT / "mst_raw"
OUT_DIR     = PROJECT_ROOT / "mst_fixed"

RAW_MST_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
def read_mst_lines(path: Path) -> list[bytes]:
    lines = []
    with path.open("rb") as f:
        for raw in f:
            line = raw.rstrip(b"\r\n")  # 레코드 한 줄에서 개행(\r\n) 제거
            if line:                    # 완전히 빈 줄은 스킵
                lines.append(line)
    if not lines:                        # 파일이 비어 있으면 바로 에러
        raise ValueError(f"{path} is empty")
    return lines

# 공통 길이 정의 (MST 파일의 고정폭 필드 길이)
SZ_SHRNCODE  = 9   # 단축코드 길이(보통 6~9자리, KRX가 9바이트로 잡아둔 필드)
SZ_STNDCODE  = 12  # 표준코드(12자리 ISIN/표준코드 계열)
SZ_KORNAME   = 40  # 한글 종목명(40바이트 고정폭)
SZ_KORNAME20 = 20  # 짧은 한글명/회원명 등(20바이트 고정폭)

# 파일별 스키마
EQUITY_SCHEMA = [
    ("short_code", SZ_SHRNCODE),  # 주식/ELW 단축코드 (예: A005930 형태, 9바이트)
    ("std_code",   SZ_STNDCODE),  # 표준코드(종목 표준코드/ISIN 계열, 12바이트)
    ("name",       SZ_KORNAME),   # 종목명(한글, 40바이트)
]

KOSPI_SCHEMA      = EQUITY_SCHEMA  # KOSPI 상장 주식 종목 마스터 구조
KOSDAQ_SCHEMA     = EQUITY_SCHEMA  # KOSDAQ 상장 주식 종목 마스터 구조
KONEX_SCHEMA      = EQUITY_SCHEMA  # KONEX 상장 주식 종목 마스터 구조
NXT_KOSPI_SCHEMA  = EQUITY_SCHEMA  # 차세대(KRX NEXT) KOSPI 종목 마스터 구조
NXT_KOSDAQ_SCHEMA = EQUITY_SCHEMA  # 차세대(KRX NEXT) KOSDAQ 종목 마스터 구조

ELW_SCHEMA = [
    ("short_code", SZ_SHRNCODE),  # ELW 단축코드
    ("std_code",   SZ_STNDCODE),  # ELW 표준코드
    ("name",       SZ_KORNAME),   # ELW 명칭
]

IDX_SCHEMA = [
    ("idx_div",  1),   # 지수 구분코드 (주가지수/채권지수/기타 등 구분용 1바이트)
    ("idx_code", 4),   # 지수 코드(4자리, 지수 식별용 코드)
    ("idx_name", 40),  # 지수명(한글, 40바이트)
]

THEME_SCHEMA = [
    ("theme_code", 3),  # 테마 코드(3자리, 테마 식별용 코드)
    ("theme_name", 40), # 테마명(한글, 40바이트)
    ("short_code", 6),  # 테마와 매핑된 대표/구성 종목 단축코드(6바이트 기준)
]

MEM_SCHEMA = [
    ("member_code", 5),        # 회원사 코드(증권사/브로커 코드, 5자리)
    ("member_name", SZ_KORNAME20),  # 회원사명(약식 한글명, 20바이트)
    ("is_global",   1),        # 글로벌 여부 플래그(해외 겸업 등 구분, 1바이트)
]

BOND_SCHEMA = [
    ("type",         2),  # 채권 유형 코드(국채/회사채/특수채 등 상위 구분, 2자리)
    ("bond_cls_code",2),  # 채권 세부 분류 코드(만기/이자구조 등 세부 분류, 2자리)
    ("std_code",     12), # 채권 표준코드(12자리)
    ("name",         40), # 채권명(한글, 40바이트)
]

FO_IDX_SCHEMA = [
    ("info_type", 1),          # 정보 타입/상품 구분 코드(선물/옵션/기타 구분, 1바이트)
    ("short_code", SZ_SHRNCODE),  # 파생상품 단축코드
    ("std_code",   SZ_STNDCODE),  # 파생상품 표준코드
    ("name",       SZ_KORNAME),   # 상품명(한글, 40바이트)
]

FO_STK_SCHEMA = FO_IDX_SCHEMA  # 주식선물/옵션 코드 구조 (지수와 동일 포맷 사용)

FO_COM_SCHEMA = [
    ("com_type",  1),          # 상품(Commodity) 타입 코드(원유/금/곡물 등, 1바이트)
    ("info_type", 1),          # 정보 타입/상품 구분 코드(선물/옵션 등, 1바이트)
    ("short_code", SZ_SHRNCODE),  # 상품 파생 단축코드
    ("std_code",   SZ_STNDCODE),  # 상품 파생 표준코드
    ("name",       SZ_KORNAME),   # 상품명(한글, 40바이트)
]

# schema 매핑 (MST 파일명 → 해당 파일의 레이아웃 정의)
SCHEMA_MAP = {
    "kospi_code.mst":      KOSPI_SCHEMA,      # 유가증권(KOSPI) 주식 종목 코드 마스터
    "nxt_kospi_code.mst":  NXT_KOSPI_SCHEMA,  # 차세대(KRX NEXT) KOSPI 종목 코드
    "kosdaq_code.mst":     KOSDAQ_SCHEMA,     # 코스닥(KOSDAQ) 주식 종목 코드 마스터
    "nxt_kosdaq_code.mst": NXT_KOSDAQ_SCHEMA, # 차세대(KRX NEXT) KOSDAQ 종목 코드
    "konex_code.mst":      KONEX_SCHEMA,      # KONEX 상장 종목 코드 마스터
    "elw_code.mst":        ELW_SCHEMA,        # ELW(주식워런트증권) 코드 마스터
    "idxcode.mst":         IDX_SCHEMA,        # 각종 지수 코드(지수 구분/코드/명칭)
    "theme_code.mst":      THEME_SCHEMA,      # 테마 코드 및 테마-종목 매핑 정보
    "memcode.mst":         MEM_SCHEMA,        # 회원사(증권사) 코드/명칭/글로벌 여부
    "bond_code.mst":       BOND_SCHEMA,       # 채권 코드(유형/분류/표준코드/명칭)
    "fo_idx_code_mts.mst": FO_IDX_SCHEMA,     # 지수선물·옵션 등 파생상품(국내) 코드
    "fo_eurex_code.mst":   FO_IDX_SCHEMA,     # 유렉스(EUREX) 파생상품 코드
    "fo_cme_code.mst":     FO_IDX_SCHEMA,     # CME 파생상품 코드
    "fo_stk_code_mts.mst": FO_STK_SCHEMA,     # 개별주식 선물·옵션 코드
    "fo_com_code.mst":     FO_COM_SCHEMA,     # 상품(Commodity) 파생상품 코드
    "fo_cmu_code.mst":     FO_COM_SCHEMA,     # 기타 상품/복합 구조 파생상품 코드(동일 레이아웃)
}


In [ ]:
def parse_fixed_width_lines(lines: list[bytes], schema, *, encoding="cp949") -> pd.DataFrame:
    record_len = max(len(l) for l in lines)
    offsets, offset = [], 0
    for name, length in schema:
        offsets.append((name, offset, length))
        offset += length

    columns = {name: [] for name, _ in schema}
    for line in lines:
        if len(line) < record_len:
            line = line.ljust(record_len, b" ")
        for name, start, length in offsets:
            raw = line[start:start+length]
            columns[name].append(raw.decode(encoding, errors="ignore").rstrip())
    return pd.DataFrame(columns)

In [ ]:
def save_df(df: pd.DataFrame, base_name: str):
    df.to_csv(OUT_DIR / f"{base_name}.csv", index=False, encoding="utf-8-sig")
    try:
        df.to_parquet(OUT_DIR / f"{base_name}.parquet", index=False)
    except:
        pass

def build_all_mst():
    out = {}
    for path in sorted(RAW_MST_DIR.glob("*.mst")):
        schema = SCHEMA_MAP.get(path.name)
        if not schema:
            continue
        lines = read_mst_lines(path)
        df = parse_fixed_width_lines(lines, schema)
        base_name = path.stem
        save_df(df, base_name)
        out[base_name] = df

    # 주식 통합 마스터
    eq = ["kospi_code", "kosdaq_code", "konex_code"]
    if all(k in out for k in eq):
        merged = pd.concat([out[k] for k in eq], ignore_index=True)
        save_df(merged, "equity_master")
        out["equity_master"] = merged

    return out

In [ ]:
all_dfs = build_all_mst()
for name, df in all_dfs.items():
    print("\n###", name)
    display(df.head())